In [ ]:
import pandas as pd

seasons = [
    '00_01', '01_02', '02_03', '03_04', '04_05',
    '05_06', '06_07', '07_08', '08_09', '09_10',
    '10_11', '11_12', '12_13', '13_14', '14_15',
    '15_16', '16_17', '17_18', '18_19', '19_20',
    '20_21', '21_22', '22_23', '23_24'
]


df = pd.DataFrame()

for season in seasons:
    df = pd.concat([df, pd.read_csv(f'../statistical_data/{season}/E0.csv', encoding='unicode_escape')])
    # df = pd.concat([df, pd.read_csv(f'./statistical_data/{season}/D1.csv')])
    # df = pd.concat([df, pd.read_csv(f'./statistical_data/{season}/I1.csv')])
    # df = pd.concat([df, pd.read_csv(f'./statistical_data/{season}/SP1.csv')])
    # df = pd.concat([df, pd.read_csv(f'./statistical_data/{season}/F1.csv')])


import matplotlib.pyplot as plt
ftr_map = {'H': 'Home Team', 'D': 'Draw', 'A': 'Away Team'}
df['FTR'] = df['FTR'].map(ftr_map)
df['FTR'].value_counts().plot.pie(autopct='%1.1f%%', startangle=90)
plt.title('Winner Distribution')
plt.axis('off')
plt.show()